# 03 – Validate SUHS‑MRV UHS Dataset

This notebook runs a few lightweight validation checks against the generated CSV files.


In [ ]:
from pathlib import Path

import pandas as pd


In [ ]:
NOTEBOOK_DIR = Path.cwd()
REPO_ROOT = NOTEBOOK_DIR.parent
DATA_DIR = REPO_ROOT / "data" / "generated"

facility_df = pd.read_csv(DATA_DIR / "facility_metadata.csv")
timeseries_df = pd.read_csv(
    DATA_DIR / "facility_timeseries.csv",
    parse_dates=["timestamp"],
)
cycle_summary_df = pd.read_csv(
    DATA_DIR / "cycle_summary.csv",
    parse_dates=["cycle_start", "cycle_end"],
)

(len(facility_df), len(timeseries_df), len(cycle_summary_df))


## Check 1 – facility_id coverage


In [ ]:
print("Unique facilities in metadata :", facility_df["facility_id"].nunique())
print("Unique facilities in timeseries:", timeseries_df["facility_id"].nunique())
print("Unique facilities in cycles    :", cycle_summary_df["facility_id"].nunique())

assert facility_df["facility_id"].nunique() == timeseries_df["facility_id"].nunique()
assert facility_df["facility_id"].nunique() == cycle_summary_df["facility_id"].nunique()
print("OK – facility_id coverage is consistent across files.")
